- age
    - younger vs. older people and decile scores, hypothesize with equivalent means
    - age-specific, using percentage of those who reoffend and see where each age compares to the mean of all people
    
- decile score

- years between offenses

- race
    - compared with decile score in more depth




In [1]:
import numpy as np
import pandas as pd

from IPython.display import Image

import warnings
warnings.simplefilter("ignore")

In [2]:
compas = pd.read_csv("Compas.csv")

In [3]:
compas.head()

,Unnamed: 0,id,first,last,sex,age,age_cat,race,juv_fel_count,decile_score,juv_misd_count,juv_other_count,priors_count,is_recid,is_violent_recid,v_decile_score,v_score_text,decile_score.1,score_text
0,0,1,miguel,hernandez,Male,69,Greater than 45,Other,0,1,0,0,0,0,0,1,Low,1,Low
1,1,2,michael,ryan,Male,31,25 - 45,Caucasian,0,5,0,0,0,-1,0,2,Low,5,Medium
2,2,3,kevon,dixon,Male,34,25 - 45,African-American,0,3,0,0,0,1,1,1,Low,3,Low
3,3,4,ed,philo,Male,24,Less than 25,African-American,0,4,0,1,4,1,0,3,Low,4,Low
4,4,5,marcu,brown,Male,23,Less than 25,African-American,0,8,1,0,1,0,0,6,Medium,8,High


In [6]:
compas.drop(compas.columns[0], axis = 1)

,id,first,last,sex,age,age_cat,race,juv_fel_count,decile_score,juv_misd_count,juv_other_count,priors_count,is_recid,is_violent_recid,v_decile_score,v_score_text,decile_score.1,score_text
0,1,miguel,hernandez,Male,69,Greater than 45,Other,0,1,0,0,0,0,0,1,Low,1,Low
1,2,michael,ryan,Male,31,25 - 45,Caucasian,0,5,0,0,0,-1,0,2,Low,5,Medium
2,3,kevon,dixon,Male,34,25 - 45,African-American,0,3,0,0,0,1,1,1,Low,3,Low
3,4,ed,philo,Male,24,Less than 25,African-American,0,4,0,1,4,1,0,3,Low,4,Low
4,5,marcu,brown,Male,23,Less than 25,African-American,0,8,1,0,1,0,0,6,Medium,8,High
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11752,11753,patrick,hamilton,Male,47,Greater than 45,Other,0,3,0,0,1,0,0,1,Low,3,Low
11753,11754,raymond,hernandez,Male,22,Less than 25,Caucasian,0,7,3,5,3,1,0,5,Medium,7,Medium
11754,11755,dieuseul,pierre-gilles,Male,35,25 - 45,Other,0,4,0,0,0,0,0,3,Low,4,Low
11755,11756,scott,lomagistro,Male,29,25 - 45,Caucasian,0,3,0,0,2,0,0,2,Low,3,Low


In [7]:
young = compas[compas.age_cat == "Less than 25"].v_decile_score
middle = compas[compas.age_cat == "25 - 45"].v_decile_score
old = compas[compas.age_cat == "Greater than 45"].v_decile_score

In [8]:
from statsmodels.stats.weightstats import ttest_ind
from scipy import stats

These tests are just checking if one age group has a higher risk of violence than the other

In [65]:
print(young.mean(), middle.mean(), old.mean())

6.049180327868853 3.3771995788840425 1.7897301349325336


In [15]:
tstat, pvalue, ddof = ttest_ind(
                                young,
                                middle,
                                alternative="larger",
                                usevar="unequal"
)
print(tstat, pvalue, ddof)

56.21398274904184 0.0 5172.47658225915


In [12]:
 tstat, pvalue, ddof = ttest_ind(
                                middle,
                                old,
                                alternative="larger",
                                usevar="unequal"
 )
print(tstat, pvalue, ddof)    

38.63847158703123 5.2422454240000335e-297 7127.205606761121


In [13]:
tstat, pvalue, ddof = ttest_ind(
                                young,
                                old,
                                alternative="larger",
                                usevar="unequal"
 )
print(tstat, pvalue, ddof) 

87.24747908351314 0.0 4723.302717729157


In [29]:
highrisk.dtypes

Unnamed: 0           int64
id                   int64
first               object
last                object
sex                 object
age                  int64
age_cat             object
race                object
juv_fel_count        int64
decile_score         int64
juv_misd_count       int64
juv_other_count      int64
priors_count         int64
is_recid             int64
is_violent_recid     int64
v_decile_score       int64
v_score_text        object
decile_score.1       int64
score_text          object
accuracy              bool
dtype: object

In [45]:
highrisk = compas[compas.score_text == "High"]
highrisk = highrisk[(highrisk.is_recid != -1) & (highrisk.decile_score != -1)]
## -1 = unknown, so those were deleted

In [46]:
highrisk["accuracy"] = (highrisk.is_recid == 1)
##checked if is_recid is True with high risk decile score to determine accuracy

In [47]:
young2 = highrisk[highrisk.age_cat == "Less than 25"].accuracy
middle2 = highrisk[highrisk.age_cat == "25 - 45"].accuracy
old2 = highrisk[highrisk.age_cat == "Greater than 45"].accuracy
##subset by age

In [48]:
print(young2.mean(), middle2.mean(), old2.mean())

0.5481927710843374 0.5325732899022801 0.4251207729468599


In [49]:
 tstat, pvalue, ddof = ttest_ind(
                                young2,
                                middle2,
                                alternative="two-sided",
                                usevar="unequal"
 )
print(tstat, pvalue, ddof) 

0.6505547751165384 0.5154436817845576 1361.6788657477787


In [51]:
 tstat, pvalue, ddof = ttest_ind(
                                young2,
                                old2,
                                alternative="two-sided",
                                usevar="unequal"
 )
print(tstat, pvalue, ddof) 

3.116051265304813 0.001986434026815587 345.5133605030412


In [52]:
 tstat, pvalue, ddof = ttest_ind(
                                middle2,
                                old2,
                                alternative="two-sided",
                                usevar="unequal"
 )
print(tstat, pvalue, ddof) 

2.882869983424161 0.004244991793179706 281.10125290631083


These hypothesis tests were checking if there was a significant difference between the age groups accuracy. 
It seems as though there is a signifcant difference between the younger two groups and the oldest group.
It would also be a good idea to look at low risk. 

In [57]:
##same procedure but with low risk
lowrisk = compas[compas.score_text == "Low"]
lowrisk = lowrisk[(lowrisk.is_recid != -1) & (lowrisk.decile_score != -1)]

In [58]:
lowrisk["accuracy"] = (lowrisk.is_recid == 0)
##accurate if is_recid == 0

In [59]:
young3 = lowrisk[lowrisk.age_cat == "Less than 25"].accuracy
middle3 = lowrisk[lowrisk.age_cat == "25 - 45"].accuracy
old3 = lowrisk[lowrisk.age_cat == "Greater than 45"].accuracy

In [60]:
print(young3.mean(), middle3.mean(), old3.mean())

0.690734055354994 0.7607060185185185 0.8178879310344828


In [64]:
 tstat, pvalue, ddof = ttest_ind(
                                young3,
                                middle3,
                                alternative="two-sided",
                                usevar="unequal"
 )
print(tstat, pvalue, ddof) 

-3.9737509848699326 7.499325893430174e-05 1192.5917089874865


In [62]:
 tstat, pvalue, ddof = ttest_ind(
                                young3,
                                old3,
                                alternative="two-sided",
                                usevar="unequal"
 )
print(tstat, pvalue, ddof) 

-6.919646610619877 6.9337001458953706e-12 1369.0461762834632


In [63]:
 tstat, pvalue, ddof = ttest_ind(
                                middle3,
                                old3,
                                alternative="two-sided",
                                usevar="unequal"
 )
print(tstat, pvalue, ddof) 

-4.958642361248988 7.383646858476191e-07 4132.705113743328


The low risk accuracy for the oldest group is much higher.
The differences in accuracy are significant between all groups. They likely do not have the same average accuracy. 
Since the average accuracy for the oldest group is significantly lower for high risk and higher for low risk, this may mean this algorithm may give the older group lower scores in general. 